<a href="https://colab.research.google.com/github/tamannashrma/GENSIM-based-Language-Models/blob/main/fgetguestcardinput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain
!pip install -U openai
!pip install -U colab-env


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for colab-env: filename=colab_env-0.2.0-py3-none-any.whl size=3806 sha256=56318f5095eb9d9c3ca1149bffaf02e91742b3dc613cdf09df47df7d8b8f12e5
  Stored in directory: /root/.cache/pip/wheels/ae/36/4f/466c2cd4db5d08f317893a920c4a0f58a81459ee3bdb136d35
Successfully built colab-env


In [2]:
import openai
import requests
import re
import json
import colab_env
import os


Mounted at /content/gdrive


In [4]:
#Define guest card function
prompt = """ You are a conversational bot for property ID '1528252253585581240' requesting information from the user (if the user wants to be contacted/called back/get in touch) -- one field at a time. The user must provide mandatory fields while and it's up to the users if they wish to provide optional fields fields are described by the following JSON Structure:
                  {
                    "Mandatory Fields": [
                      "firstName",
                      "lastName",
                      "email",

                    ],
                    "Optional Fields": [
                    "desiredBedrooms",
                    "desiredMoveIn date",
                    "desiredPriceLow",
                    "desiredPriceHigh",
                    "phone number"
                    ]
                  }
                  Extract details from the user inputs.

                  Ask the user the questions needed to gather the required fields.

                  - For the name, request both the first name and last name. If they provide both in the response, parse them into first and last names. If the user enters a keyboard mash in the name like 'asdf' or 'qwerty', request that they provide it again or confirm the name.

                  - Assume that any 10-digit number that looks like a phone number, which the user provides, is a phone number.

                  - For the bedroom type, if the user specifies 'studio', please assume 0. A property can have any number of bedrooms.

                  - Convert the user-provided move-in date to the MM/DD/YYYY format.

                  - If a single amount is given for rent, convert it to a range by adding and subtracting 500 from the amount.

                  - The propertyId is always '1528252253585581240'.

                  - Accept questions in comments. Any additional information that the user provides that's not captured in other fields should go into comments.

                  - For dates, assume today's date is June 8, 2023.

                  - If the user gives input as 'n/a', assume it's missing.

                  Convert the gathered information into JSON before confirming the final summary.

                  Here's an example JSON output:
                  {
                    "propertyId": "",
                    "firstName": "",
                    "lastName": "",
                    "email": "",
                    "phone number": "",
                    "desiredBedrooms": "",
                    "desiredMoveIn date": "",
                    "desiredPriceLow": "",
                    "desiredPriceHigh": "",
                    "comment": "",
                    "source": ""
                  }

                  If any value in the JSON is blank, 'n/a', missing, "", or None, please remove the key-value from the JSON.

                  Once you have gathered the required information, ask the user if they wish to add anything to the JSON structure.
                  You can provide a summary of the gathered information in JSON format.

                  For example:
                  "Based on your inputs, here's a summary of the gathered information:
                  {
                    "firstName": "John",
                    "lastName": "Doe",
                    "email": "johndoe@example.com",
                    "phone number": "+1234567890",
                    "desiredBedrooms": "2",
                    "desiredMoveIn date": "07/15/2023",
                    "desiredPriceLow": "$2000",
                    "desiredPriceHigh": "$2500",
                    "comment": "Additional comments here",
                    "source": "Chatbot"
                  }

                  Strictly output the JSON only.
                  """
def gc_func():
  class ChatApp:
      def __init__(self):
          openai.api_key = os.getenv("OPENAI_API_KEY")

      userprompt = """ Everytime a User inputs, extract information from that input and check for the fields that have not ben provided and ask User to provide those fields one-by-one.
      """
      messages = [
          {"role": "system", "content": prompt},
         {"role": "user", "content": userprompt},
           ]

      def chat(self, message):
          self.messages.append({"role": "user", "content": message})
          response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo-0613",
              temperature=0.1,
              messages=self.messages
          )
          self.messages.append({"role": "assistant", "content": response["choices"][0]["message"].content})
          return response["choices"][0]["message"]["content"]

  app = ChatApp()
  json_done=False
  while json_done == False:
      res = app.chat(input())
      print(res)
      try:
        my_json = re.findall("\{[\s\S]*\}", res)[0]
        json_done = True
      except:
        pass
      if "}" in res:
        json_done = True

  return my_json

In [5]:
gc_func()

Someone cal me back
Sure, I can help you with that. Could you please provide me with your first name?
Tamna
Thank you for providing your first name, Tamna. Could you also please provide me with your last name?
Sharma
Thank you for providing your last name, Sharma. Could you please provide me with your email address?
Tama22@gmail.com
Thank you for providing your email address, Tama22@gmail.com. Could you please provide me with your phone number?
9999999999
Thank you for providing your phone number, 9999999999. 

Now, let's move on to the optional fields. Could you please provide me with the number of desired bedrooms? If you are looking for a studio apartment, please enter 0.
1
Thank you for providing the number of desired bedrooms as 1. 

Next, could you please provide me with your desired move-in date? Please enter it in the format MM/DD/YYYY.
22/11/2023
Thank you for providing your desired move-in date as 22/11/2023. 

Now, could you please provide me with your desired price range fo

'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Tamna",\n  "lastName": "Sharma",\n  "email": "Tama22@gmail.com",\n  "phone number": "9999999999",\n  "desiredBedrooms": "1",\n  "desiredMoveIn date": "11/22/2023",\n  "desiredPriceHigh": "$20000"\n}'

In [7]:
gc_func()

My name is Tashi Mishra and call me
Thank you for providing your name, Tashi Mishra. Could you please provide your email address?
tashi @gmail.com
Thank you for providing your email address, tashi@gmail.com. Could you please provide your phone number?
n/a
Thank you for letting me know. Could you please provide the number of desired bedrooms?
n/a
Thank you for letting me know. Could you please provide your desired move-in date?
n/a
Thank you for letting me know. Could you please provide your desired price range?
20000
Thank you for providing the desired price range. Just to confirm, is the price range $20,000?
yes
Thank you for confirming. Could you please provide any additional comments or information you would like to include?
No
Thank you for providing all the necessary information. Here's a summary of the gathered information:

{
  "propertyId": "1528252253585581240",
  "firstName": "Tashi",
  "lastName": "Mishra",
  "email": "tashi@gmail.com",
  "desiredPriceLow": "$19500",
  "desi

'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Tashi",\n  "lastName": "Mishra",\n  "email": "tashi@gmail.com",\n  "desiredPriceLow": "$19500",\n  "desiredPriceHigh": "$20500"\n}'

In [ ]:
gc_func()

Interested in the property but have questions about the amenities. Can someone call me at 9160102134? My name is Ethan Thompson and my email is ethan@example.com.
Based on your inputs, here's a summary of the gathered information:
{
  "firstName": "Ethan",
  "lastName": "Thompson",
  "email": "ethan@example.com",
  "telephone": "9160102134",
  "source": "Chatbot"
}

Is there anything else you would like to add?


'{\n  "firstName": "Ethan",\n  "lastName": "Thompson",\n  "email": "ethan@example.com",\n  "telephone": "9160102134",\n  "source": "Chatbot"\n}'

In [3]:
gc_func()

NameError: ignored

In [ ]:
gc_func()

How do I contact the property owner or agent? My name is Ava Roberts and my email is ava@example.com.
Thank you for providing your contact information, Ava Roberts. To better assist you, could you please provide your phone number as well?
9999999999
Thank you for providing your phone number, Ava Roberts. 

Based on the information you provided, here's a summary of the gathered information:

{
  "propertyId": "1528252253585581240",
  "firstName": "Ava",
  "lastName": "Roberts",
  "email": "ava@example.com",
  "telephone": "9999999999"
}

Is there anything else you would like to add?


'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Ava",\n  "lastName": "Roberts",\n  "email": "ava@example.com",\n  "telephone": "9999999999"\n}'

In [ ]:
gc_func()

I am 'Munish Gandhi', email 'mg@hy.ly', I would like to move in by '1st July, 2023' and prefer a one bedroom apartment
Great! I have gathered some information from you. Here's a summary of the gathered information:

{
  "firstName": "Munish",
  "lastName": "Gandhi",
  "email": "mg@hy.ly",
  "desiredBedrooms": "1",
  "desiredMoveIn": "07/01/2023"
}

Is there anything else you would like to add?


'{\n  "firstName": "Munish",\n  "lastName": "Gandhi",\n  "email": "mg@hy.ly",\n  "desiredBedrooms": "1",\n  "desiredMoveIn": "07/01/2023"\n}'

In [ ]:
gc_func()

I am Rama Challa, email 'Rama@hy.ly', I would like to move in by '1st July, 2023' and prefer a studio apartment
Based on your inputs, here's a summary of the gathered information:
{
  "firstName": "Rama",
  "lastName": "Challa",
  "email": "Rama@hy.ly",
  "desiredBedrooms": "0",
  "desiredMoveIn": "07/01/2023",
  "source": "Chatbot"
}

Is there anything else you would like to add?


'{\n  "firstName": "Rama",\n  "lastName": "Challa",\n  "email": "Rama@hy.ly",\n  "desiredBedrooms": "0",\n  "desiredMoveIn": "07/01/2023",\n  "source": "Chatbot"\n}'

In [ ]:
gc_func()

Can someone reach out to me? I'm available from 9 am to 2 pm on weekdays. My name is Benjamin Davis and my email is benjamin@example.com.
Sure, I can help you with that. Just to confirm, your name is Benjamin Davis and your email is benjamin@example.com, correct?
Yes
Great! Thank you for confirming. 

To assist you further, I need a few more details. Could you please provide me with your phone number?
9999999999
Thank you for providing your phone number. 

Now, could you please let me know the number of bedrooms you are looking for? If you are looking for a studio apartment, please enter 'studio'.
1
Thank you for providing the number of bedrooms you are looking for.

Next, could you please provide me with your desired move-in date? If you don't have a specific date in mind, you can enter 'n/a'.
21/12/2023
Thank you for providing your desired move-in date. 

Lastly, could you please provide me with your desired price range for the property? If you have a specific budget in mind, please 

'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Benjamin",\n  "lastName": "Davis",\n  "email": "benjamin@example.com",\n  "telephone": "9999999999",\n  "desiredBedrooms": "1",\n  "desiredMoveIn": "12/21/2023",\n  "desiredPriceLow": "$1500",\n  "desiredPriceHigh": "$2500"\n}'

In [ ]:
gc_func()

Can someone give me a call? My name is Emily Anderson, and my email is emily.anderson@example.com.{ 
Based on your inputs, here's a summary of the gathered information:
{
  "firstName": "Emily",
  "lastName": "Anderson",
  "email": "emily.anderson@example.com",
  "source": "Chatbot"
}

Is there anything else you would like to add?


'{\n  "firstName": "Emily",\n  "lastName": "Anderson",\n  "email": "emily.anderson@example.com",\n  "source": "Chatbot"\n}'

In [ ]:
gc_func()

How can I get in touch with you? My name is Sophia Davis, and my email is sophia.davis@example.com.
Thank you for providing your name and email, Sophia Davis. To better assist you, may I have your phone number as well?
99899998989
Thank you for providing your phone number, Sophia Davis. 

Based on the information you've provided, here's a summary of the gathered information:

{
  "firstName": "Sophia",
  "lastName": "Davis",
  "email": "sophia.davis@example.com",
  "telephone": "99899998989"
}

Is there anything else you would like to add?


'{\n  "firstName": "Sophia",\n  "lastName": "Davis",\n  "email": "sophia.davis@example.com",\n  "telephone": "99899998989"\n}'

In [ ]:
gc_func()

I'm looking for more information. Can someone call me back? My name is Olivia Davis, and my email is olivia.davis@example.com
Sure, I can help you with that. Could you please provide me with your phone number?
999999999
Thank you for providing your phone number. Is there anything else you would like to add to the information?
No
Based on the information you provided, here's a summary of the gathered information:

{
  "propertyId": "1528252253585581240",
  "firstName": "Olivia",
  "lastName": "Davis",
  "email": "olivia.davis@example.com",
  "telephone": "999999999",
  "source": "Chatbot"
}

Is there anything else you would like to add?


'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Olivia",\n  "lastName": "Davis",\n  "email": "olivia.davis@example.com",\n  "telephone": "999999999",\n  "source": "Chatbot"\n}'

In [ ]:
gc_func()

Can I request a call from your team? My name is Mickel Muse and my budget is $1000 USD.
Sure, I can help you with that. Before we proceed, may I have your email address?
mickey@gmail.com
Thank you for providing your email address. May I also have your phone number?
9999999999
Thank you for providing your phone number. Just to confirm, your name is Mickel Muse, your budget is $1000 USD, and your email address is mickey@gmail.com. Is that correct?
yes
Great! Based on the information you provided, here's a summary of the gathered information:

{
  "propertyId": "1528252253585581240",
  "firstName": "Mickel",
  "lastName": "Muse",
  "email": "mickey@gmail.com",
  "telephone": "9999999999",
  "desiredPriceLow": "$1000",
  "desiredPriceHigh": "$1500",
  "source": "Chatbot"
}

Is there anything else you would like to add?


'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Mickel",\n  "lastName": "Muse",\n  "email": "mickey@gmail.com",\n  "telephone": "9999999999",\n  "desiredPriceLow": "$1000",\n  "desiredPriceHigh": "$1500",\n  "source": "Chatbot"\n}'

In [ ]:
gc_func()

I'd like to speak with someone about the available apartments. Can you give me a call? My name is William , and my email is william.johnson@example.com.
Sure, I can assist you with that. Before we proceed, may I have your last name, please?
Jhonsan
Thank you for providing your last name, William Jhonsan. I have noted it down. 

To assist you further, may I have your phone number?
9999999999
Thank you for providing your phone number, William Jhonsan. 

To better understand your requirements, could you please let me know the number of bedrooms you are looking for?
1
Thank you for providing the number of bedrooms you are looking for, William Jhonsan. 

Next, could you please provide me with your desired move-in date?
n/a
Understood, William Jhonsan. If you don't have a specific move-in date, we can proceed without it. 

Now, could you please provide me with your desired price range for the apartment?
$4000
Thank you for providing your desired price range, William Jhonsan. 

Lastly, is the

'{\n  "propertyId": "1528252253585581240",\n  "firstName": "William",\n  "lastName": "Jhonsan",\n  "email": "william.johnson@example.com",\n  "telephone": "9999999999",\n  "desiredBedrooms": "1",\n  "desiredPriceLow": "$3500",\n  "desiredPriceHigh": "$4500",\n  "source": "Chatbot"\n}'

In [ ]:
gc_func()

Can you provide me with your contact information? My name is Sophia Thompson, and my email is sophia.thompson@example.com.
Thank you for providing your contact information, Sophia Thompson. I have your first name as Sophia and your last name as Thompson. Is that correct?
yes
Great! I have your first name as Sophia and your last name as Thompson. 

Is there anything else you would like to add to the information?
No
Alright, based on the information provided, here's a summary of the gathered information:

{
  "propertyId": "1528252253585581240",
  "firstName": "Sophia",
  "lastName": "Thompson",
  "email": "sophia.thompson@example.com"
}

Is there anything else you would like to add?


'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Sophia",\n  "lastName": "Thompson",\n  "email": "sophia.thompson@example.com"\n}'

In [ ]:
gc_func()

I have some questions about the property. Can someone reach out to me? My name is Jacob Wilson, and my email is jacob.wilson@example.com.
Sure, I can help you with that. Before we proceed, may I know your phone number?
999999999
Thank you for providing your phone number. 

Based on the information you provided, here's a summary of the gathered information:

{
  "firstName": "Jacob",
  "lastName": "Wilson",
  "email": "jacob.wilson@example.com",
  "telephone": "999999999",
  "source": "Chatbot"
}

Is there anything else you would like to add?


'{\n  "firstName": "Jacob",\n  "lastName": "Wilson",\n  "email": "jacob.wilson@example.com",\n  "telephone": "999999999",\n  "source": "Chatbot"\n}'

In [ ]:
gc_func()

I would like to discuss the available units. Can you call me back? My name is Olivia Smith, and my email is olivia.smith@example.com.
Sure, I can help you with that. Before we proceed, may I have your phone number, please?
999999999
Thank you for providing your phone number. I will make sure to include it in the information. Is there anything else you would like to add?
No
Based on the information you provided, here's a summary of the gathered information:

{
  "propertyId": "1528252253585581240",
  "firstName": "Olivia",
  "lastName": "Smith",
  "email": "olivia.smith@example.com",
  "telephone": "999999999",
  "source": "Chatbot"
}

Is there anything else you would like to add?


'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Olivia",\n  "lastName": "Smith",\n  "email": "olivia.smith@example.com",\n  "telephone": "999999999",\n  "source": "Chatbot"\n}'

In [ ]:
gc_func()

Can you give me a call? My name is William Johnson, and my email is william.johnson@example.com and my budget is $4000.
Sure, I can give you a call. Before that, could you please provide me with your phone number?
999999999
Thank you for providing your phone number. I will give you a call shortly. Just to confirm, your name is William Johnson and your email is william.johnson@example.com. Is that correct?
Yes
Great! I have gathered the following information:

- First Name: William
- Last Name: Johnson
- Email: william.johnson@example.com
- Phone Number: 999999999
- Desired Price Range: $3500 - $4500 (converted from $4000)

Based on your inputs, here's a summary of the gathered information:
{
  "firstName": "William",
  "lastName": "Johnson",
  "email": "william.johnson@example.com",
  "telephone": "999999999",
  "desiredPriceLow": "$3500",
  "desiredPriceHigh": "$4500",
  "source": "Chatbot"
}

Is there anything else you would like to add?


'{\n  "firstName": "William",\n  "lastName": "Johnson",\n  "email": "william.johnson@example.com",\n  "telephone": "999999999",\n  "desiredPriceLow": "$3500",\n  "desiredPriceHigh": "$4500",\n  "source": "Chatbot"\n}'

In [ ]:
gc_func()

Can I get a call back? My name is Robert Johnson and my email is robert@example.com.
Sure, I can help you with that. May I have your phone number, please?
9898765788
Thank you for providing your phone number. Is there anything else you would like to add to the information?
I want to ask about amenties 
Thank you for your interest in the property. I will make a note to inquire about the amenities for you. Is there anything else you would like to add to the information?
No
Based on the information you provided, here's a summary of the gathered information:

{
  "propertyId": "1528252253585581240",
  "firstName": "Robert",
  "lastName": "Johnson",
  "email": "robert@example.com",
  "telephone": "9898765788",
  "source": "Chatbot"
}

Is there anything else you would like to add?


'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Robert",\n  "lastName": "Johnson",\n  "email": "robert@example.com",\n  "telephone": "9898765788",\n  "source": "Chatbot"\n}'

In [ ]:
gc_func()

How can I reach you to discuss this matter further? My name is Samantha Brown and my email is samantha@example.com.
Thank you for providing your name and email, Samantha Brown. To ensure I have your contact information correctly, could you please confirm your full name and email address?
yes
Great! Here's a summary of the gathered information so far:

{
  "firstName": "Samantha",
  "lastName": "Brown",
  "email": "samantha@example.com"
}

Is there anything else you would like to add?


'{\n  "firstName": "Samantha",\n  "lastName": "Brown",\n  "email": "samantha@example.com"\n}'

In [ ]:
gc_func()

Please let me know the best way to contact you. My name is Matthew Davis and my email is matthew@example.com.
Thank you for providing your contact information, Matthew. To better assist you, may I know your phone number as well?
9898765678
Thank you for providing your phone number, Matthew. 

Based on the information you provided, here's a summary of the gathered information:

{
  "propertyId": "1528252253585581240",
  "firstName": "Matthew",
  "lastName": "Davis",
  "email": "matthew@example.com",
  "telephone": "9898765678"
}

Is there anything else you would like to add?


'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Matthew",\n  "lastName": "Davis",\n  "email": "matthew@example.com",\n  "telephone": "9898765678"\n}'

In [ ]:
gc_func()

Hi, my name is John Smith and my email is john@example.com. I'm interested in a two-bedroom apartment with a price range of $1500 to $2000.
Great, John! I have gathered some information from you. Let me confirm what I have so far:

{
  "firstName": "John",
  "lastName": "Smith",
  "email": "john@example.com",
  "desiredBedrooms": "2",
  "desiredPriceLow": "$1500",
  "desiredPriceHigh": "$2000"
}

Is there anything else you would like to add?


'{\n  "firstName": "John",\n  "lastName": "Smith",\n  "email": "john@example.com",\n  "desiredBedrooms": "2",\n  "desiredPriceLow": "$1500",\n  "desiredPriceHigh": "$2000"\n}'

In [ ]:
gc_func()

Can someone call me back? My name is Jennifer Smith and my email is jennifer@example.com.
Sure, I can help you with that. Before we proceed, may I know your phone number?
234598765
Thank you for providing your phone number. To confirm, your name is Jennifer Smith, your email is jennifer@example.com, and your phone number is 234598765. Is that correct?
yes
Great! Here's a summary of the gathered information:

{
  "propertyId": "1528252253585581240",
  "firstName": "Jennifer",
  "lastName": "Smith",
  "email": "jennifer@example.com",
  "telephone": "234598765"
}

Is there anything else you would like to add?


'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Jennifer",\n  "lastName": "Smith",\n  "email": "jennifer@example.com",\n  "telephone": "234598765"\n}'

In [ ]:
gc_func()

Hello, my name is John Smith. My email address is 'johnsmith@example.com'. I want to move in by 'August 15th, 2023' and I'm looking for a 2-bedroom apartment
Great! I have gathered some information from you. Here's a summary of what I have so far:

{
  "firstName": "John",
  "lastName": "Smith",
  "email": "johnsmith@example.com",
  "desiredBedrooms": "2",
  "desiredMoveIn": "08/15/2023"
}

Is there anything else you would like to add?


'{\n  "firstName": "John",\n  "lastName": "Smith",\n  "email": "johnsmith@example.com",\n  "desiredBedrooms": "2",\n  "desiredMoveIn": "08/15/2023"\n}'

In [ ]:
gc_func()

Call me - 9160102134. My name is Olivia Wilson, my price range is below $4000 and my email is olivia@example.com.
Thank you for providing your information. Just to confirm, your name is Olivia Wilson, your phone number is 9160102134, your email is olivia@example.com, and your desired price range is below $4000. Is that correct?
Yes
Great! Here's a summary of the gathered information:

{
  "propertyId": "1528252253585581240",
  "firstName": "Olivia",
  "lastName": "Wilson",
  "email": "olivia@example.com",
  "telephone": "9160102134",
  "desiredPriceLow": "$3500",
  "desiredPriceHigh": "$4500"
}

Is there anything else you would like to add?


'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Olivia",\n  "lastName": "Wilson",\n  "email": "olivia@example.com",\n  "telephone": "9160102134",\n  "desiredPriceLow": "$3500",\n  "desiredPriceHigh": "$4500"\n}'

In [ ]:
gc_func()

Hi, I'm Jennifer Wilson, and I'm looking for both 1-bedroom and 2-bedroom apartments. You can reach me at 'jennifer.wilson@example.com'. What are my options? I plan to move in on 'July 15th, 2023'
Based on your inputs, here's a summary of the gathered information:
{
  "firstName": "Jennifer",
  "lastName": "Wilson",
  "email": "jennifer.wilson@example.com",
  "desiredBedrooms": "1,2",
  "desiredMoveIn": "07/15/2023"
}

Is there anything else you would like to add?


'{\n  "firstName": "Jennifer",\n  "lastName": "Wilson",\n  "email": "jennifer.wilson@example.com",\n  "desiredBedrooms": "1,2",\n  "desiredMoveIn": "07/15/2023"\n}'

In [ ]:
gc_func()

My name is James Anderson, and my email is 'james.anderson@example.com'. I'm interested in a studio apartment. My preferred move-in date is 'November 1st, 2023'.
Great! I have gathered some information from you. Here's a summary of what I have so far:

{
  "firstName": "James",
  "lastName": "Anderson",
  "email": "james.anderson@example.com",
  "desiredBedrooms": "0",
  "desiredMoveIn": "11/01/2023"
}

Is there anything else you would like to add?


'{\n  "firstName": "James",\n  "lastName": "Anderson",\n  "email": "james.anderson@example.com",\n  "desiredBedrooms": "0",\n  "desiredMoveIn": "11/01/2023"\n}'

In [ ]:
gc_func()

 Hello, I'm Emily Davis. You can reach me at 'emily.davis@gmail.com'. I'd like to move in on 'July 15th, 2023'. I'm open to both 1-bedroom and 2-bedroom apartments.
Based on your inputs, here's a summary of the gathered information:
{
  "firstName": "Emily",
  "lastName": "Davis",
  "email": "emily.davis@gmail.com",
  "desiredMoveIn": "07/15/2023",
  "desiredBedrooms": "1 or 2",
  "source": "Chatbot"
}

Is there anything else you would like to add?


'{\n  "firstName": "Emily",\n  "lastName": "Davis",\n  "email": "emily.davis@gmail.com",\n  "desiredMoveIn": "07/15/2023",\n  "desiredBedrooms": "1 or 2",\n  "source": "Chatbot"\n}'

In [ ]:
gc_func()

Can someone reach out to me? I'm available 9 am-2 pm on weekdays.
Sure, I can help you with that. Before we proceed, I need to gather some information from you. Let's start with your first name.
Tamy
Thank you for providing your first name, Tamy. Now, may I have your last name, please?
koki
Thank you for providing your last name, Tamy Koki. Now, I need your email address to contact you. Could you please provide your email?
kokila@gmail.com
Thank you for providing your email address, Tamy Koki. To better assist you, I also need your phone number. Could you please provide your phone number?
999999999
Thank you for providing your phone number, Tamy Koki. 

Now, let's move on to the optional fields. If you have any specific requirements for the number of bedrooms, please let me know. If not, you can simply type 'n/a'.
1
Thank you for letting me know that you are looking for 1 bedroom. 

Next, could you please provide me with your desired move-in date? If you don't have a specific date in m

'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Tamy",\n  "lastName": "Koki",\n  "email": "kokila@gmail.com",\n  "telephone": "999999999",\n  "desiredBedrooms": "1",\n  "desiredMoveIn": "n/a",\n  "desiredPriceLow": "$1500",\n  "desiredPriceHigh": "$2500"\n}'

In [ ]:
gc_func()

How do I contact the property owner or agent?
To contact the property owner or agent, I will need some information from you. Let's start with your first name.
Ram Tara
Thank you for providing your first name, Ram Tara. May I have your last name, please?
Ram tara Bargail
Thank you for providing your last name, Ram Tara Bargail. Could you please provide me with your email address?
gogo@gg.com
Thank you for providing your email address, gogo@gg.com. 

Now, I need to gather some optional information. If you would like to provide any of the following details, please let me know:

- Desired number of bedrooms
- Desired move-in date
- Desired price range (low and high)
- Phone number

Please let me know which details you would like to provide, or if you have any additional comments or questions.
No
Alright, no problem. 

Based on the information you have provided, here's a summary of the gathered information:

{
  "propertyId": "1528252253585581240",
  "firstName": "Ram Tara",
  "lastName": "

'{\n  "propertyId": "1528252253585581240",\n  "firstName": "Ram Tara",\n  "lastName": "Bargail",\n  "email": "gogo@gg.com"\n}'

In [ ]:
gc_func()

Can someone call me at 9160102134
Sure, I will note down your phone number as 9160102134. Is there anything else you would like to add?
No
Based on your inputs, here's a summary of the gathered information:
{
  "telephone": "9160102134"
}

Is there anything else you would like to add?


'{\n  "telephone": "9160102134"\n}'